In [1]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
# Step 1: Load and Preprocess Data

# Dataset directories (replace with your paths)
with_mask_dir = "input/Mask_Detection/with_mask"
without_mask_dir = "input/Mask_Detection/without_mask"

data = []
labels = []

# Load images and assign labels
for category, label in [(with_mask_dir, 0), (without_mask_dir, 1)]:
    for file in os.listdir(category):
        img_path = os.path.join(category, file)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (128, 128))
        data.append(image)
        labels.append(label)

# Convert to numpy arrays
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)

In [5]:
data.shape

(7553, 128, 128, 3)

In [6]:
labels.shape

(7553,)

In [7]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

In [8]:
# Step 2: Data Augmentation
train_aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [9]:
# Step 3: Build the Model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model layers

model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(2, activation="softmax")
])

model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [10]:
# Step 4: Train the Model
history = model.fit(
    train_aug.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20
)

Epoch 1/20


E:\ML\ML_Models\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


189/189 ━━━━━━━━━━━━━━━━━━━━ 51s 233ms/step - accuracy: 0.8896 - loss: 0.6887 - val_accuracy: 0.9729 - val_loss: 0.0740
Epoch 2/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 43s 226ms/step - accuracy: 0.9534 - loss: 0.1271 - val_accuracy: 0.9795 - val_loss: 0.0673
Epoch 3/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 43s 222ms/step - accuracy: 0.9517 - loss: 0.1221 - val_accuracy: 0.9788 - val_loss: 0.0527
Epoch 4/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 42s 221ms/step - accuracy: 0.9636 - loss: 0.1125 - val_accuracy: 0.9815 - val_loss: 0.0557
Epoch 5/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 42s 221ms/step - accuracy: 0.9638 - loss: 0.0988 - val_accuracy: 0.9835 - val_loss: 0.0457
Epoch 6/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 43s 222ms/step - accuracy: 0.9672 - loss: 0.1086 - val_accuracy: 0.9828 - val_loss: 0.0450
Epoch 7/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 45s 238ms/step - accuracy: 0.9728 - loss: 0.0836 - val_accuracy: 0.9861 - val_loss: 0.0404
Epoch 8/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 42s 222ms/step - accuracy: 0.9689 - loss: 0.0879 - val

In [11]:
# Step 5: Evaluate the Model
y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred, target_names=["With Mask", "Without Mask"]))

48/48 ━━━━━━━━━━━━━━━━━━━━ 8s 147ms/step
              precision    recall  f1-score   support

   With Mask       0.97      0.99      0.98       745
Without Mask       0.99      0.98      0.98       766

    accuracy                           0.98      1511
   macro avg       0.98      0.98      0.98      1511
weighted avg       0.98      0.98      0.98      1511



In [13]:
# Step 6: Save the Model
model.save("face_mask_detector.keras")

In [2]:
# Step 7: Real-Time Detection with OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [3]:
# Load trained model
from tensorflow.keras.models import load_model
model = load_model("face_mask_detector.keras")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (128, 128))
        face = np.expand_dims(face, axis=0) / 255.0

        prediction = model.predict(face)
        label = "Mask" if np.argmax(prediction) == 0 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Face Mask Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━